In [70]:
import nltk
import re
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# Importação do arquivo

In [71]:
#Ajustar tamanho máximo de linha visível. Max caracteres por tweet: 280
pd.set_option('max_colwidth', 300)

In [72]:
#Importar dados do dataset encoding incompatível com utf-8
data = pd.read_csv("train.csv", encoding='ISO-8859-1')
#Remover colunas desncessárias
data = data.drop(columns=['ItemID','Sentiment'])

In [73]:
#Criar coluna com o tamanho de caracteres de cada tweet
data['length'] = [len(tweet[0]) for tweet in data.values]

In [74]:
#Linhas com mais e menos caracteres
data.sort_values(by='length', ascending=False)

,SentimentText,length
8834,"brokeback mountain was terrible.\n8838,0,Sentiment140,# @Catherine42 I wouldn't mind but I only had 1/2 a portion &amp; then left 1/2 the cream just fruit for me then until my hols x\n8839,1,Sentiment140,# @DeliciousLunch ... dark chocolate cookies? oh you tease! I'm writing to day n dipping ...",949
11346,"#self_info 2ë¬ë? ê°ë°ì?... ì?¸ê°ì ?ì?¸ ê°ë°ì?ë¥¼ ê¿ê¿ëë¤. íì¬ ì§ì­ê´ê³ ìë²ë¥¼ íê³ , ê°ì?¸ì ?ì¼ë¡ë ê²ììì§ì? ë§ë¤ë ¤ê³ ì¤ë¹ì¤ì ëë¤. ì¤ë¹. . .ã ã ã  íë¡ì° ê³ ê³ ê³ ~ ê¸°ì ìì? ì¬ëìì§ ì¬ëìì? ê¸°ì ìë¤~!",274
33208,@ahohey project design à¹à¸à¹à¸?à¸à¸µà¹à¸à¸µà¸à¸µà¹à¸¡à¸«à¹à¸­à¸­à¸?à¹?à¸à¸à¹à¸£à¸à¸à¸¢à¸²à¸à¸²à¸¥....à¸?à¸³à¸¥à¸±à¸à¸à¸³à¹à¸à¸£à¹?à¸?à¸£à¸¡à¸­à¸¢à¸¹à¹....à¹à¸à¸´à¸?à¹à¸à¸¢à¸¡à¸² comment à¸à¸µà¸?à¸§à¹à¸² 5555 function à¹à¸¢à,255
9495,"&quot;Ð¯ Ð¿Ð¾Ð´Ð¿Ð¸Ñ?Ð°Ð»Ñ?Ñ? Ð½Ð° Ð²Ð°ÑÑ RSS Ð»ÐµÐ½ÑÑ, Ð½Ð¾ Ñ?Ð¾Ð¾Ð±ÑÐµÐ½Ð¸Ñ? Ð¿Ð¾ÑÐµÐ¼Ñ-ÑÐ¾ Ð² Ð²Ð¸Ð´Ðµ ÐºÐ°ÐºÐ¸Ñ -ÑÐ¾ ÐºÐ²Ð°Ð´ÑÐ°ÑÐ¾Ð² ÐÐ°Ðº Ñ?ÑÐ¾ Ð¸Ñ?Ð¿ÑÐ°Ð²Ð¸ÑÑ?&quot; ÐÐ¾Ð¹ÑÐ¸ Ð½Ð°Ñ Ð¸ Ð·Ð°Ð±ÑÑÑ Ð¿ÑÐ¾ Ð¼Ð¾Ñ RSS ))",244
46373,@ApartHotel ÑÑ Ñ?ÐºÐ¾ÑÐµÐµ Ð¿Ð¾ÑÐµÑÑ?ÐµÑÑ Ð¸Ð»Ð¸ Ð¿Ð¾Ð¼ÐµÐ½Ñ?ÐµÑÑ ÑÐµÐ»ÐµÑÐ¾Ð½ Ð¿ÑÐµÐ¶Ð´Ðµ ÑÐµÐ¼ Ñ?Ð´Ð¾Ñ Ð½ÐµÑ Ð°ÐºÐºÑÐ¼ÑÐ»Ñ?ÑÐ¾Ñ ÐÐ°ÑÐ¾ Ð¿ÑÐ¾ÑÐ½Ð¾Ñ?ÑÑ Ñ?Ð±Ð¾ÑÐºÐ¸ Ð±ÐµÑ?Ð¿Ð¾Ð´Ð¾Ð±Ð½Ð°Ñ? - ÐºÐ°Ðº Ð¼Ð¾Ð½Ð¾Ð»Ð¸Ñ!,237
...,...,...
34061,@ajbee,7
12256,*heart,7
4192,yeesh,7
2014,FOOD!,7


In [ ]:
#Dados sobre a quantidade de caracteres
data.describe()

## Contagem de emoticons
fonte: https://www.kaggle.com/youben/twitter-sentiment-analysis

In [75]:
#Contagem (com alterações)
tweets_text = data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?\S) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)

[(3281, ':/'),
 (1339, 'x@'),
 (1214, 'xx'),
 (1162, 'xa'),
 (984, ';3'),
 (887, 'xp'),
 (842, 'xo'),
 (713, ';)'),
 (483, 'xe'),
 (431, ';I'),
 (353, ';.'),
 (254, 'xD'),
 (251, 'x.'),
 (245, '::'),
 (217, ';t'),
 (209, ';s'),
 (185, ':O'),
 (176, ':3'),
 (166, ';D'),
 (159, ":'"),
 (157, 'XD'),
 (146, 'x3'),
 (142, ':p'),
 (126, ":'("),
 (118, ':@'),
 (117, 'xh'),
 (117, ':S'),
 (109, 'xm'),
 (104, ';p'),
 (104, ';-)'),
 (92, ':|'),
 (91, 'x,'),
 (89, ';P'),
 (76, 'xd'),
 (75, ';o'),
 (75, ';d'),
 (71, ':o'),
 (65, 'XX'),
 (63, ':L'),
 (59, 'Xx'),
 (59, ':1'),
 (58, ':]'),
 (57, ':s'),
 (56, ':0'),
 (54, 'XO'),
 (44, ';;'),
 (43, ';('),
 (38, ':-D'),
 (37, 'xk'),
 (36, 'XT'),
 (35, 'x?'),
 (35, 'x)'),
 (34, 'x2'),
 (33, ';/'),
 (32, 'x:'),
 (32, ':\\'),
 (31, 'x-'),
 (27, 'Xo'),
 (27, 'XP'),
 (27, ':-/'),
 (26, ':-P'),
 (25, ':*'),
 (23, 'xX'),
 (22, ":')"),
 (17, 'xP'),
 (16, ':['),
 (16, ':-p'),
 (14, 'x]'),
 (14, 'XM'),
 (13, ':-O'),
 (12, 'x('),
 (12, 'X1'),
 (12, ':x'),
 (11, 'X

In [ ]:
#Busca (com alterações)
HAPPY_EMO = r" ([xX;:]-?[bdD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[xXsS/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

# Palavras mais usadas
fonte: https://www.kaggle.com/youben/twitter-sentiment-analysis

In [ ]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [ ]:
most_used_words(data.SentimentText.str.cat())[:100] #100 mais frequentes

In [ ]:
#Mais frequentes sem stopwords
mw = most_used_words(data.SentimentText.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

In [64]:
most_words[-15:]

['box',
 'radio',
 'kidding',
 'iphone',
 'air',
 'minute',
 'warm',
 'power',
 'country',
 'test',
 'flight',
 'YES',
 'OK',
 'Chris',
 'LMAO']

## Stemming

In [52]:
#nltk.download('wordnet')
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

In [68]:
stemmed = stem_tokenize(" ".join(most_words))

In [69]:
stemmed[-15:]

['box',
 'radio',
 'kidding',
 'iphone',
 'air',
 'minute',
 'warm',
 'power',
 'country',
 'test',
 'flight',
 'YES',
 'OK',
 'Chris',
 'LMAO']

# Preparação do texto

In [88]:
#Limpeza de texto 01
def clean_text_round1(text):
    text = text.lower().strip()
    text = re.sub(r'@\w+\s?', ' ', text) #Remover nomes de usuários
    text = re.sub(r'https?://.+\S', ' ', text) #Remover links
    text = re.sub('&[\w*%s*]' % re.escape(string.punctuation), ' ', text) #Apagar '&' e caracteres posteriores 
    text = re.sub(r'#', ' ', text) #Remove hashtags
    text = re.sub('\s{2,}', '', text) #Remover excesso de espaços
    text = re.sub(r'\s', ' ', text) #Remover whitespaces    
    return text

round1 = lambda x: clean_text_round1(x)

In [89]:
#Criar novo dataframe
clean = pd.DataFrame.from_dict(data.SentimentText.apply(round1)) 

In [90]:
#Criar coluna com o tamanho de caracteres de cada texto
clean['length'] = [len(text[0]) for text in clean.values]

In [81]:
#Salvar arquivo
clean.to_csv('clean.csv')

In [79]:
clean.describe()
clean

,SentimentText,length
0,is so sad for my apl friend.............,40
1,i missed the new moon trailer...,32
2,omg its already 7:30 :o,23
3,.. omgaga. im soooim gunna cry. i've been at this dentist since 11.. i was suposed 2 just get a crown put on (30mins)...,120
4,i think mi bf is cheating on me!!!t_t,37
...,...,...
99984,seems like a repeating problemhope you're able to find something.,65
99985,"arrrr we both replied to each other over different tweets at the same time, i'll see you then, duno where the hell kateyy is!",126
99986,ya i thought so,16
99987,yes. yes. i'm glad you had more fun with me.,45


In [91]:
#Limpeza de texto 02
def clean_text_round2(text):
    #text = re.sub(r'\W', " ", text)
    text = re.sub(r'(.)\1{3,}', r'\1', text) #Remove repetições MELHORAR
    return text

round2 = lambda x : clean_text_round2(x)

In [93]:
clean.SentimentText = pd.DataFrame.from_dict(clean.SentimentText.apply(round2))
clean

,SentimentText,length
0,is so sad for my apl friend.,40
1,i missed the new moon trailer...,32
2,omg its already 7:30 :o,23
3,.. omgaga. im soooim gunna cry. i've been at this dentist since 11.. i was suposed 2 just get a crown put on (30mins)...,120
4,i think mi bf is cheating on me!!!t_t,37
...,...,...
99984,seems like a repeating problemhope you're able to find something.,65
99985,"ar we both replied to each other over different tweets at the same time, i'll see you then, duno where the hell kateyy is!",126
99986,ya i thought so,16
99987,yes. yes. i'm glad you had more fun with me.,45


In [85]:
#Remover linhas com menos de 2 caracteres ou mais de 300
new = np.array([[line[0], int(line[1])] for line in clean.values if 300 > line[1] > 2])
new = pd.DataFrame.from_dict(new)
new.columns = ['text', 'length']

In [86]:
to_int = lambda x: int(x)
new['length'] = new.length.apply(to_int)

In [87]:
#Salvar data
new.to_csv("new_data.csv")

In [ ]:
new.sort_values(by='length', ascending=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(new.text.head(10000))
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = new.head(10000).index
data_cv

In [ ]:
print(data_dtm)